# NLP Quick Start for newbie😁 - with 9 steps

## INDEX
```
Step 1. Library Import & Data Load
Step 2. Data Preprocessing
     2-a. Drop Columns
     2-b. Tokenizer
     2-c. Pad Sequences
     2-d. Match Data type to numpy.ndarray
Step 3. Modeling
Step 4. Model Compile
Step 5. Callbacks
Step 6. Model Fit
Step 7. Model Evaluate & Save
Step 8. Reload Model
Step 9. Predict Test Data
```
---

## Step 1. Library Import & Data Load

In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.nunique()

In [ ]:
test_df.head()

In [ ]:
test_df.info()

In [ ]:
test_df.nunique()

## Step 2. Data Preprocessing

### 2-a. Drop Columns

In [ ]:
train_df.head()

In [ ]:
train_df.drop(columns=['id','keyword','location'], axis=1, inplace=True)

In [ ]:
test_df.head()

In [ ]:
test_df.drop(columns=['keyword','location'],axis=1, inplace=True)

In [ ]:
print(train_df.shape, test_df.shape)

### 2-b. Tokenizer

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df['text'],train_df['target'], test_size=0.2, random_state=111)

In [ ]:
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
vocab_size = 1000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)

In [ ]:
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_valid = tokenizer.texts_to_sequences(X_valid)

In [ ]:
for i in range(10):
    print(len(X_train[i]))

In [ ]:
X_train[0]

In [ ]:
for i in range(10):
    print(len(X_valid[i]))

In [ ]:
X_valid[0]

### 2-c. Pad Sequences

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_length = 120
trunc_type = 'post'
pad_type = 'post'

In [ ]:
X_train_padded = pad_sequences(X_train, maxlen=max_length, truncating=trunc_type, padding=pad_type)
X_valid_padded = pad_sequences(X_valid, maxlen=max_length, truncating=trunc_type, padding=pad_type)

In [ ]:
X_train_padded[:2]

In [ ]:
X_valid_padded[:2]

In [ ]:
print(X_train_padded.shape, X_valid_padded.shape)

### 2-d. Match Data type to numpy.ndarray

In [ ]:
print(type(X_train_padded), type(X_valid_padded))
print(type(y_train), type(y_valid))

In [ ]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)

In [ ]:
print(type(X_train_padded), type(X_valid_padded))
print(type(y_train), type(y_valid))

## Step 3. Modeling

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten

In [ ]:
embedding_dim = 16
# vocab_size = 1000
# max_length = 120

In [ ]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64, dropout=0.5)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

## Step 4. Model Compile

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

## Step 5. Callbacks

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
filepath = 'my_checkpoint.ckpt'
cp = ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

In [ ]:
ep = EarlyStopping(
    monitor='val_loss', 
    patience=5,
)

## Step 6. Model Fit

In [ ]:
epochs=30
model.fit(
    X_train_padded, y_train,
    validation_data = (X_valid_padded, y_valid),
    callbacks=[cp,ep],
    epochs=epochs
)

## Step 7. Model Evaluate & Save

In [ ]:
model.load_weights(filepath)

In [ ]:
model.evaluate(X_valid_padded, y_valid)

In [ ]:
X_valid[0]

In [ ]:
model.save('./model/basic_nlp.h5')

## Step 8. Reload Model

In [ ]:
import tensorflow as tf

In [ ]:
mymodel = tf.keras.models.load_model('./model/basic_nlp.h5')

In [ ]:
mymodel.summary()

## Step 9. Predict Test Data 

In [ ]:
X_test = tokenizer.texts_to_sequences(test_df['text'])

In [ ]:
X_test_padded = pad_sequences(X_test, maxlen=max_length, truncating=trunc_type, padding=pad_type)

In [ ]:
y_test_raw = model.predict(X_test_padded)

In [ ]:
y_test_raw

In [ ]:
y_test = list(map(lambda x : 1 if x > 0.5 else 0, y_test_raw))

In [ ]:
set(y_test)

In [ ]:
y_test[:5]

In [ ]:
test_df['predict'] = y_test

In [ ]:
test_df

In [ ]:
test_df[test_df['predict']==1]

In [ ]:
submission = test_df[['id','predict']]

In [ ]:
submission

In [ ]:
submission.columns = ['id', 'target']

In [ ]:
submission

In [ ]:
submission.to_csv('./sample_submission.csv', index=False)